<a href="https://colab.research.google.com/github/XGHouFTW/py-cryptopredict/blob/main/StockPriceGaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
transMonth = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}

start = "2020-01-31" #from historical prices
end = "2022-01-31" #732 days
def trans(strdate):
    return datetime.datetime(int(strdate[0:4]), int(strdate[5:7]), int(strdate[8:10]))

In [3]:
cryptopricekey = ["BTC","ETH","DOGE"]
cryptobasepath = "/content/drive/MyDrive/ISEF/Datasets/Historical Prices/" # + cryptopricekey + "-USD.csv"
stockpricekey = ["AAPL","AMC","DJI","GME","NASDAQ","S&P","TSLA"]
stockbasepath = "/content/drive/MyDrive/ISEF/Datasets/Stonks/" # + stockpricekey + "_historical.csv"

searchkey = ["bitcoin","ethereum","dogecoin","cryptocurrency","economy","finance","politics","pandemic"]
dfPrice = pd.DataFrame()
dfAddDate = pd.read_csv("/content/drive/MyDrive/ISEF/Datasets/Historical Prices/BTC-USD.csv")
dfPrice["Date"] = dfAddDate["Date"].apply(lambda text: trans(text))

In [4]:
for word in cryptopricekey:
  dfAdd = pd.read_csv(cryptobasepath + word + "-USD.csv")
  del dfAdd["Date"]
  dfAdd = dfAdd.rename(columns=lambda x: word + "-" + x)
  dfPrice = dfPrice.join(dfAdd)

,Date,BTC-Open,BTC-High,BTC-Low,BTC-Close,BTC-Adj Close,BTC-Volume,ETH-Open,ETH-High,ETH-Low,ETH-Close,ETH-Adj Close,ETH-Volume,DOGE-Open,DOGE-High,DOGE-Low,DOGE-Close,DOGE-Adj Close,DOGE-Volume
0,2020-01-31,9508.313477,9521.706055,9230.776367,9350.529297,9350.529297,29432489719,184.736908,185.405838,176.296814,180.160172,180.160172,11728616394,0.002440,0.002458,0.002369,0.002394,0.002394,76519295
1,2020-02-01,9346.357422,9439.323242,9313.239258,9392.875000,9392.875000,25922656496,180.113770,183.845551,179.745178,183.673950,183.673950,11569697182,0.002391,0.002431,0.002382,0.002422,0.002422,78730830
2,2020-02-02,9389.820313,9468.797852,9217.824219,9344.365234,9344.365234,30835736946,183.532501,193.080399,180.173950,188.617538,188.617538,14054425389,0.002421,0.002506,0.002395,0.002468,0.002468,91552515
3,2020-02-03,9344.683594,9540.372070,9248.633789,9293.521484,9293.521484,30934096509,188.607407,193.436890,188.012695,189.865067,189.865067,12392875241,0.002466,0.002517,0.002427,0.002440,0.002440,78856631
4,2020-02-04,9292.841797,9331.265625,9112.811523,9180.962891,9180.962891,29893183716,189.861725,191.111496,185.403625,189.250595,189.250595,11714191695,0.002438,0.002503,0.002402,0.002503,0.002503,89790002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2022-01-27,36841.878906,37148.324219,35629.281250,37138.234375,37138.234375,25041426629,2467.188477,2510.451660,2328.452148,2423.001221,2423.001221,16126522783,0.143756,0.145010,0.136947,0.141247,0.141247,608707586
728,2022-01-28,37128.445313,37952.878906,36211.109375,37784.332031,37784.332031,22238830523,2421.646973,2548.778564,2367.833740,2547.092041,2547.092041,14666227351,0.141244,0.142413,0.138380,0.141656,0.141656,506667466
729,2022-01-29,37780.714844,38576.261719,37406.472656,38138.179688,38138.179688,17194183075,2546.590576,2627.609375,2526.989746,2597.084717,2597.084717,11172062661,0.141649,0.143984,0.141480,0.143056,0.143056,397610776
730,2022-01-30,38151.917969,38266.339844,37437.710938,37917.601563,37917.601563,14643548444,2598.564941,2631.408447,2550.460938,2603.466553,2603.466553,9501221177,0.143057,0.143515,0.138754,0.139471,0.139471,363976741


In [6]:
for word in stockpricekey:
  dfAdd = pd.read_csv(stockbasepath + word + "_historical.csv")
  


SyntaxError: ignored